In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import string
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/Data/"

In [4]:
df = pd.read_csv(data_path+"Data_clean_processed_sample.csv")

In [ ]:
df.head()

,email_id,subject,email,bow,embeddings
0,saibi-e\inbox\612#1,WSCC (Mahave 02) (San Juan 03),"Dear Power Outage Database Customer,\n\nAttach...",dear power outage database customer attached f...,"dear power outage database customer, attached ..."
1,kitchen-l\_americas\netco_restart\28#2,RE:,Ryan is waffling. I talked to him this weeken...,ryan waffling talked weekend said wife want mo...,ryan waffling. talked weekend said wife want m...
2,motley-m\sent_items\1#1,SSARR Update Available Now,"<center><LI><a Href=""http://www.nwrfc.noaa.gov...",columbia river basin lower snake updated mar,columbia river basin lower snake.....updated: mar
3,kaminski-v\all_documents\577#1,Credit Risk Model Comments - at this point.,"Hi everyone,\n\nI have run the model and, alon...",everyone run model along contract brief questi...,"everyone, run model and, along contract brief ..."
4,quigley-d\sent_items\480#1,Confirmation: Risk Management Simulation Mee...,"Hi Dutch, This message is to confirm ou...",dutch message confirm meeting thursday novembe...,"dutch, message confirm meeting on, thursday, n..."


In [5]:
# !pip install hlda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install tomotopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.5 MB 14.1 MB/s 


In [7]:
import tomotopy

In [8]:
from tomotopy.utils import Corpus

corpus = Corpus()
j = 1
for i in df['bow']:
  corpus.add_doc(i.split(), a_data=j)
  j+=1
# corpus.add_doc("e f g h i".split(), a_data=2)
# corpus.add_doc("i j k l m".split(), a_data=3)

In [9]:
mdl = tomotopy.HLDAModel(min_cf=25, min_df=25, rm_top=0, depth=2, alpha=0.1, eta=0.01, gamma=0.1, corpus=corpus, transform=None)

In [11]:
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)

Num docs: 20000 , Vocab size: 0 , Num words: 0


In [ ]:
for _ in range(0, 20000, 100):
    mdl.train(70)
    mdl.train(30, freeze_topics=True)
    print('Iteration: {:05}\tll per word: {:.5f}\tNum. of topics: {}'.format(corpus.global_step, corpus.ll_per_word, corpus.live_k))

# for _ in range(0, 2000, 10):
#     a.train(10, freeze_topics=True)
#     print('Iteration: {:05}\tll per word: {:.5f}\tNum. of topics: {}'.format(a.global_step, a.ll_per_word, a.live_k))

In [ ]:
# I haven't tried this code before.
test_result_cps, ll = mdl.infer(corpus)
for doc in test_result_cps:
    print(doc.path, doc.get_words(top_n=10))

Another package: HLDA.sampler

In [ ]:
from hlda.sampler import HierarchicalLDA

In [ ]:
vocab = []
for sent in df['bow']:
  vocab+=sent.split()
vocab = sorted(set(vocab))

In [ ]:
vocab_index = {}
for i, w in enumerate(vocab):
    vocab_index[w] = i

In [ ]:
new_corpus = []
for doc in df['bow']:
    doc = doc.split()
    new_doc = []
    for word in doc:
        word_idx = vocab_index[word]
        new_doc.append(word_idx)
    new_corpus.append(new_doc)

In [ ]:
n_samples = 10       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 3   # the number of iterations between printing a brief summary of the topics so far
n_words = 5           # the number of most probable words to print for each topic after model estimation
with_weights = False

In [ ]:
hlda = HierarchicalLDA(new_corpus, vocab, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling

... 3
topic=0 level=0 (documents=20000): please, enron, would, know, get, 
    topic=1 level=1 (documents=6697): enron, please, would, mail, may, 
        topic=2 level=2 (documents=1627): power, energy, california, said, state, 
        topic=7 level=2 (documents=47): folder, synchronizing, item, offline, added, 
        topic=8 level=2 (documents=3405): please, schedule, agreement, know, thanks, 
        topic=16 level=2 (documents=1092): imagemasker, way, one, urlmasker, new, 
        topic=21 level=2 (documents=53): epmi, term, total, deal, short, 
        topic=31 level=2 (documents=231): time, description, imagemasker, central, standard, 
        topic=43 level=2 (documents=19): schneider, tirrell, tribolet, hate, harry, 
        topic=47 level=2 (documents=24): cold, plus, alka, seltzer, medicine, 
        topic=69 level=2 (documents=23): pcbs, equipment, decontamination, surface, solvent, 
        topic=96 level=2 (documents=23): btu, mag, weinart, wol